In [1]:
import $ivy.`org.apache.spark::spark-sql:2.4.3` 
import org.apache.spark.sql._
import org.apache.log4j.{Level, Logger}
Logger.getLogger("org").setLevel(Level.OFF)
import org.joda.time.format.DateTimeFormat
import org.joda.time.{DateTime, Days}
import org.joda.time.format.DateTimeFormat
import java.util.Properties
import org.apache.spark.sql.DataFrame
import org.apache.spark.sql.{DataFrame, SQLContext}
import org.apache.spark.{SparkConf, SparkContext}
import org.joda.time.{DateTime, Days}
import org.apache.spark.sql.{DataFrame, SparkSession}
import org.apache.spark.sql.SaveMode._
import org.apache.spark.sql.expressions.Window
import org.apache.spark.SparkConf
import org.apache.spark.sql.SparkSession
import org.json4s._
import org.json4s.jackson.JsonMethods._
import org.joda.time.format.DateTimeFormat
import org.joda.time.{DateTime, Days}
val formatter = DateTimeFormat.forPattern("yyyy-MM-dd")
import scala.util.Try
import java.sql.{Connection, DriverManager, ResultSet}
import org.apache.spark.SparkContext  
import org.apache.spark.SparkConf
import org.apache.spark.sql.Column
import org.apache.spark.sql.types.{DataType, DateType, TimestampType}
import org.apache.spark.sql.functions._
val spark = {
  SparkSession.builder()
    .master("local[*]")
    .getOrCreate()
}
def sc = spark.sparkContext
import spark.implicits._

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties


import $ivy.$                                   

import org.apache.spark.sql._

import org.apache.log4j.{Level, Logger}

import org.joda.time.format.DateTimeFormat

import org.joda.time.{DateTime, Days}

import org.joda.time.format.DateTimeFormat

import java.util.Properties

import org.apache.spark.sql.DataFrame

import org.apache.spark.sql.{DataFrame, SQLContext}

import org.apache.spark.{SparkConf, SparkContext}

import org.joda.time.{DateTime, Days}

import org.apache.spark.sql.{DataFrame, SparkSession}

import org.apache.spark.sql.SaveMode._

import org.apache.spark.sql.expressions.Window

import org.apache.spark.SparkConf

import org.apache.spark.sql.SparkSession

import org.json4s._

import org.json4s.jackson.JsonMethods._

import org.joda.time.format.DateTimeFormat

import org.joda.time.{DateTime, Days}

formatter: org.joda.time.format.DateTimeFormatter = org.joda.time.format.DateTimeFormatter@5c76eacd
import scala.util.Try

import java.sql.{Connection, DriverManager, ResultSe

In [78]:
var frame5 = spark.read.option("header", true).option("inferSchema",true).csv("apply_beautify.csv")
var frame = frame5.select("country_code", "timestamp", "device_id", "event_type", "session_id", "user_id", "market", "tool", 
                        "height", "width")
frame.printSchema()
frame.filter(col("timestamp")==="2019-02-03 04:00:53.0").show(3)

root
 |-- country_code: string (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- device_id: string (nullable = true)
 |-- event_type: string (nullable = true)
 |-- session_id: string (nullable = true)
 |-- user_id: long (nullable = true)
 |-- market: string (nullable = true)
 |-- tool: string (nullable = true)
 |-- height: integer (nullable = true)
 |-- width: integer (nullable = true)

+------------+---------+---------+----------+----------+-------+------+----+------+-----+
|country_code|timestamp|device_id|event_type|session_id|user_id|market|tool|height|width|
+------------+---------+---------+----------+----------+-------+------+----+------+-----+
+------------+---------+---------+----------+----------+-------+------+----+------+-----+



frame5: DataFrame = [Accept-Encoding: string, advertising_id: string ... 20 more fields]
frame: DataFrame = [country_code: string, timestamp: timestamp ... 8 more fields]

In [57]:
//1.Opens with faces photos: All?
frame.agg(countDistinct(col("device_id"))).show()

+-------------------------+
|count(DISTINCT device_id)|
+-------------------------+
|                    11868|
+-------------------------+



In [68]:
//2.How many times face tool was applied.   
//either tool primarily named "face"
frame.filter(col("tool").contains("face")).agg(countDistinct(col("device_id")).as("count_tools_containing_face")).show()
// or tool connected with face parts, for example "teeth_whiten". In this case all tools correspond except "null" and "none"
frame.filter(col("tool")=!="null" || col("tool")=!="none").agg(countDistinct(col("device_id")).
                                            as("count_tools_except_null_and_none")).show()

+---------------------------+
|count_tools_containing_face|
+---------------------------+
|                       1490|
+---------------------------+

+--------------------------------+
|count_tools_except_null_and_none|
+--------------------------------+
|                           11844|
+--------------------------------+



In [91]:
//import scala.math
//3.Change (%) in subscribed users' share ?
// find min date of the dataframe
def subscription_share_difference(frame:DataFrame):Unit = {
    var frame_1 = frame.withColumn("date", to_date(frame("timestamp")))
var min_date = frame_1.select("date").agg(min(col("date"))).take(1)(0)(0).toString
//find number of subscribed users and number of device ids (as total users), and calculate share of subscribed users
var num_subs_users = frame_1.filter(col("date")===min_date).agg(countDistinct("user_id")).take(1)(0)(0).toString.toDouble
var num_total_users = frame_1.filter(col("date")===min_date).agg(countDistinct("device_id")).take(1)(0)(0).toString.toDouble
var share_sub_users_initial = num_subs_users*100/num_total_users
/** all distinct users who did any action on any day after the first day are actual subscribed users (would be more correct
if we had the data about the users who unsubscribed on those days)*/
var num_subs_users_now = frame_1.filter(col("date")>min_date).agg(countDistinct("user_id")).take(1)(0)(0).toString.toDouble
var num_total_users_now = frame_1.filter(col("date")>min_date).agg(countDistinct("device_id")).take(1)(0)(0).toString.toDouble
var share_sub_users_now = num_subs_users_now*100/num_total_users_now
var percent_change_in_shares = (share_sub_users_now-share_sub_users_initial)/share_sub_users_initial*100
println("percent change in subscribed users' share is "+ scala.math.round(percent_change_in_shares)+" percent")
}
subscription_share_difference(frame)

percent change in subscribed users' share is 9 percent


defined function subscription_share_difference

In [62]:
//4.beautify open-apply conversion (have only apply data)
//5.stickiness for subscribed users only
def stickiness(dataframe:DataFrame):Double = {
        var frame = dataframe.withColumn("date", to_date(dataframe("timestamp"))).
                    withColumn("month", month(dataframe("timestamp")))
        var stickiness_raw = avg_DAU(frame)*100/avg_MAU(frame)       
        var stickiness = scala.math.round(stickiness_raw)
        stickiness
    } 
def avg_DAU(dataframe:DataFrame):Double = {
        var frame = dataframe.withColumn("date", to_date(col("timestamp")))
        var avg_DAU_frame = group_count(frame, Array("date"), Array("user_id")).
                            agg(round(avg(col("count_user_id"))))  
        var avg_dau = avg_DAU_frame.take(1)(0)(0).toString.toDouble
        avg_dau
    }
def avg_MAU(dataframe:DataFrame):Double = {
        var frame = dataframe.withColumn("date", to_date(dataframe("timestamp"))).
                    withColumn("month", month(dataframe("timestamp")))
        var avg_MAU_frame = group_count(frame, Array("month"), Array("user_id")).
                            agg(round(avg(col("count_user_id"))))  
        var avg_mau = avg_MAU_frame.take(1)(0)(0).toString.toDouble
        avg_mau
    }
def group_count(dataframe:DataFrame, group_col:Array[String], count:Array[String]):DataFrame = {
        var count_col:Array[Column] = count.map(c => countDistinct(col(c)).as("count_"+c))
        var grouped_frame = dataframe.groupBy(group_col.head, group_col.tail:_*).agg(count_col.head, count_col.tail:_*)
        grouped_frame
    }
stickiness(frame)

defined function stickiness
defined function avg_DAU
defined function avg_MAU
defined function group_count
res61_4: Double = 25.0

In [92]:
/**6.Engagement: the more sessions a daily users does during a day, the more engaged he/she is, so we need to divide 
average number of sessions per day by average number of daily active users*/

def engagement(dataframe:DataFrame):Double = {
        var frame = dataframe.withColumn("date", to_date(dataframe("timestamp")))
        var avg_dau = avg_DAU(frame)
        var frame_1 = group_count(frame, Array("date"), Array("session_id")).
                      agg(round(avg(col("count_session_id"))))
        var sessions_num = frame_1.take(1)(0)(0).toString.toDouble
        var engagement = scala.math.round(sessions_num/avg_dau)
        engagement
    }
engagement(frame)


defined function engagement
res91_1: Double = 2.0

In [96]:
//7.most used tools combinataions
def most_used_tools_combinations(dataframe:DataFrame):Unit = {
        
        var window = Window.partitionBy("session_id").orderBy("timestamp")
        var new_df = dataframe.
        withColumn("tool_2", lead("tool", 1) over window).
        withColumn("tools_comb", concat(col("tool"),lit(","),col("tool_2"))).select("session_id","tools_comb").
        groupBy("tools_comb").agg(countDistinct("session_id").as("number_of_users")).orderBy(col("number_of_users").desc).
        withColumn("percent_of_all_users",round(col("number_of_users")/dataframe.agg(countDistinct("session_id")).
        take(1)(0)(0).toString.toInt*100,2)).show(7, false)
    }
most_used_tools_combinations(frame)

+---------------------+---------------+--------------------+
|tools_comb           |number_of_users|percent_of_all_users|
+---------------------+---------------+--------------------+
|null                 |11906          |100.0               |
|skin_tone,skin_tone  |3              |0.03                |
|eye_color,eye_color  |2              |0.02                |
|hair_color,details   |2              |0.02                |
|smooth,teeth_whiten  |2              |0.02                |
|hair_color,hair_color|2              |0.02                |
|details,face_fix     |1              |0.01                |
+---------------------+---------------+--------------------+
only showing top 7 rows



defined function most_used_tools_combinations

In [98]:
//8. applies per tool frequencies [subscribed users]
group_count(frame, Array("tool"), Array("user_id")).show()

+---------------+-------------+
|           tool|count_user_id|
+---------------+-------------+
|      skin_tone|         1011|
|       face_fix|          857|
|           none|            1|
|           auto|          543|
|      eye_color|          431|
|           null|           10|
|     hair_color|          829|
|   teeth_whiten|          333|
|red_eye_removal|          132|
|         smooth|          775|
|        details|          791|
|  auto_beautify|           31|
|    blemish_fix|          950|
+---------------+-------------+

